# Azure AI Search CSV integrated vectorization sample

Based on: [https://github.com/Azure/azure-search-vector-samples/blob/main/demo-python/code/indexers/csv.ipynb](https://github.com/Azure/azure-search-vector-samples/blob/main/demo-python/code/indexers/csv.ipynb)

Demonstrates use of generative technology in both steps of a typical RAG pattern: data retrieval and prompt response.

For data retrieval, the LLM is instructed to generate a search query that is optimized to retrun only the data needed by the user prompt and use available features
of the index, e.g. filters, sort options and response count (top). The returned data is then provided to a separate LLM completion request to answer the user's prompt.

### Prerequisites

+ An Azure subscription, with [access to Azure OpenAI](https://aka.ms/oai/access).
 
+ Azure AI Search, any tier, but we recommend Basic or higher for this workload. [Enable semantic ranker](https://learn.microsoft.com/azure/search/semantic-how-to-enable-disable) if you want to run a hybrid query with semantic ranking.

+ A deployment of the `text-embedding-3-large` model on Azure OpenAI.

+ A deployment of the `gpt-4o` model on Azure OpenAI. 

+ Azure Blob Storage. This notebook connects to your storage account and loads a container with the sample CSV.


### Set up a Python virtual environment in Visual Studio Code

1. Open the Command Palette (Ctrl+Shift+P).
1. Search for **Python: Create Environment**.
1. Select **Venv**.
1. Select a Python interpreter. Choose 3.10 or later.

It can take a minute to set up. If you run into problems, see [Python environments in VS Code](https://code.visualstudio.com/docs/python/environments).

### Install packages

In [1]:
! pip install -r requirements.txt --quiet

### Load .env file (Copy .env-sample to .env and update accordingly)

In [1]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

load_dotenv(override=True) # take environment variables from .env.

endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
credential = AzureKeyCredential(os.getenv("AZURE_SEARCH_ADMIN_KEY")) if os.getenv("AZURE_SEARCH_ADMIN_KEY") else DefaultAzureCredential()
index_name = os.getenv("AZURE_SEARCH_INDEX", "csv-vec")
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_key = os.getenv("AZURE_OPENAI_KEY")
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
azure_openai_model_name = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL_NAME", "text-embedding-3-large")
azure_openai_model_dimensions = int(os.getenv("AZURE_OPENAI_EMBEDDING_DIMENSIONS", 1024))
azure_openai_chat_deployment = os.getenv("AZURE_OPENAI_CHATGPT_DEPLOYMENT", "gpt-4o")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-05-01-preview")

# Comment out when NOT using APIM
azure_openai_endpoint=os.environ["APIM_AZURE_OPENAI_ENDPOINT"]
azure_openai_key=os.getenv("APIM_AZURE_OPENAI_KEY")


### Define index schema

Used later on to constrain which fields are used for sorting or filtering.

In [2]:
import os
from azure.search.documents.indexes.models import (
    SearchField,
    SearchFieldDataType,
)

fields = [  
    SearchField(name="AzureSearch_DocumentKey",  key=True, type=SearchFieldDataType.String),
    SearchField(name="ID", type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=False),  
    SearchField(name="Name", type=SearchFieldDataType.String, filterable=True),  
    SearchField(name="Age", type=SearchFieldDataType.Int32, sortable=True, filterable=True, facetable=False),  
    SearchField(name="Title", type=SearchFieldDataType.String, sortable=False, filterable=False, facetable=False),
    SearchField(name="Description", type=SearchFieldDataType.String, sortable=False, filterable=False, facetable=False),
    SearchField(name="TitleVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), vector_search_dimensions=azure_openai_model_dimensions, vector_search_profile_name="myHnswProfile"),
    SearchField(name="DescriptionVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), vector_search_dimensions=azure_openai_model_dimensions, vector_search_profile_name="myHnswProfile"),
]
searchable_fields = ",".join([field.name for field in fields if field.searchable])
sortable_fields = ",".join([field.name for field in fields if field.sortable])
filterable_fields = ",".join([field.name for field in fields if field.filterable])  

### Use LLM to convert user prompt to a search expression

#### Create OpenAI client

In [3]:
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

openai_credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(openai_credential, "https://cognitiveservices.azure.com/.default")

client = AzureOpenAI(
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key,
    azure_ad_token_provider=token_provider if not azure_openai_key else None
)

#### Test it

In [4]:
response = client.chat.completions.create(
    model=azure_openai_chat_deployment,
    messages=[
        {"role": "system", "content": "You are thoughtfull assistant answering silly questions"},
        {"role": "user", "content": "What color is the sky?"}
    ]
)
print(response.choices[0].message)

ChatCompletionMessage(content="The color of the sky can vary depending on several factors, but it's most commonly blue during the daytime due to the scattering of sunlight by the atmosphere. This phenomenon is known as Rayleigh scattering, where shorter blue wavelengths are scattered in all directions more than other colors. During sunrise or sunset, the sky can appear red, orange, or pink because the sun's light passes through more of the Earth's atmosphere, scattering shorter wavelengths and allowing the longer red and orange wavelengths to dominate. At night, the sky is generally dark but can be dotted with the light from stars and other celestial bodies.", refusal=None, role='assistant', function_call=None, tool_calls=None)


#### Qury processing client

Some questions require a deeper understanding of the data schema. For example, the question "Which employees are older than 40?" requires using [filtering](https://learn.microsoft.com/en-us/azure/search/search-filters) and "Who is the youngest employee" requires using [sorting](https://learn.microsoft.com/en-us/azure/search/search-pagination-page-layout). Use your [chat deployment](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/completions) to create the correct Azure Search query to answer the question.

Using 'function calling (tools)' as a way to have OpenAI generate a json object, rather than json text in a markdown response format. See this [article](https://platform.openai.com/docs/guides/structured-outputs/function-calling-vs-response-format) for options.

In [6]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
import pandas as pd
import json

query_options_system_prompt = f"""
Given a user prompt, search for pertinent data using. 
"""
sortDescription = f"""
Specify a custom sort order for search results. Format is a comma-separated list of up to 32 order-by clauses. An order-by clause consists of a field name to order by and optional direction.
You must generate one clause for every field you want to sort by. 
If a direction is not specified, the default is ascending. 
Examples:
Query: Find the youngest employee
Response: Age asc

Query: Who is the youngest, tallest employee
Response: Height desc, Age asc
"""

# Note that 'call_search' is a non-existent function used here so that OpenAI returns correctly formatted json object (otherwise, it returns a markdown string with a json embedded)
tools = [
    {
        "type": "function",
        "function": {
            "name": "call_search",
            "description": "Given a question, get any additional Azure Search query parameters required to answer the question. If no additional query parameters are required to answer the question, don't return any.",
            "parameters": {
                "type": "object",
                "properties": {
                    "orderBy": {
                        "type": "string",
                        "description": sortDescription,
                    },
                    "filter": {
                        "type": "string",
                        "description": f"Specify inclusion or exclusion criteria for search results. Format is an Azure Search OData boolean expression. Example: Age le 4 or not (Age gt 8). You may only use the following fields: {filterable_fields}"
                    },
                    "search": {
                        "type": "string",
                        "description": "The search property is a string that specifies a query string used to search text and vectors in an Azure Search index in order to answer the provided question. If no query string is required to answer the question, return * or no query string at all"
                    },
                    "top": {
                        "type": "integer",
                        "description": "The number of search results to return. The default is 50. The maximum is 1000. If less than 50 are needed to satisfy the query, set this field to the needed number."
                    },
                }
            },
        }
    }
]

# NOTE: the tools are used so that the function callings are not included in the response message and the message contains properly format json object
def get_query_options(query: str) -> dict:
    response = client.chat.completions.create(
        model=azure_openai_chat_deployment,
        messages=[
            {"role": "system", "content": query_options_system_prompt},
            {"role": "user", "content": query}
        ],
        tools=tools,
        tool_choice={ "type": "function", "function": { "name": "call_search" } },
    )
    response_message = response.choices[0].message

    # Only include query options if the model provides them
    if len(response_message.tool_calls) == 1:
        try:
            return json.loads(response_message.tool_calls[0].function.arguments)
        except:
            return {}

    return {}



### Search and complete

Convert the returned search options into appropriate parameters to Azure Search. Verify that returned sort fields are defined as sortable. If not. raise an error - the user query cannot be answered efficiently unless the field is sortable.

In [15]:
search_client = SearchClient(endpoint, index_name, credential=credential)
def answer_query(query: str) -> str:
    # Parse the query options returned by the model
    query_options = get_query_options(query)
    query_option_search = query_options.get("search")
    vector_queries = None
    if query_option_search and query_option_search != "*":
        vector_queries = [VectorizableTextQuery(text=query_option_search, k_nearest_neighbors=50, fields="TitleVector,DescriptionVector")]

    query_option_order_by = query_options.get("orderBy")
    order_by = None
    if query_option_order_by:
        order_by = query_option_order_by.split(",")
        # foreach order_by, check whether it starts with a valid field name
        # if not, throw an exception that the field is not sortable
        for order in order_by:
            field = order.strip().split(" ")[0]
            if field not in [field.name for field in fields if field.sortable]:
                # Consider saving this info in audit trail to identify fields which should be made sortable
                raise Exception(f"Data search cannot be completed as field '{field}' is not sortable field in the index and is needed by the prompt.")

    # This sample only uses specific fields to answer questions. Update these fields for your own data
    columns = ["ID", "Age", "Name", "Title", "Description"]
    search_results = search_client.search(
        search_text=query_option_search,
        vector_queries=vector_queries,
        top=query_options.get("top", 5),
        order_by=order_by,
        filter=query_options.get("filter"),
        select=columns
    )

    # Convert the search results to markdown for use by the model
    results = [ { column: result[column] for column in columns } for result in search_results ]
    results_markdown_table = pd.DataFrame(results).to_markdown(index=False)

    response = client.chat.completions.create(
        model=azure_openai_chat_deployment,
        messages=[
            {"role": "system", "content": f"The following question requires search results to provide an answer. Use the provided search results to answer the question. If you can't answer the question using the search results, say I don't know."},
            {"role": "user", "content": results_markdown_table },
            {"role": "user", "content": query}
        ]
    )
    # Return the generated answer, query options, and results table for analysis
    return response.choices[0].message.content, query_options, results_markdown_table

def print_answer(answer, query_options, results):
    print("Generated Query Options:", query_options)
    print("Search Results")
    print(results)
    print("Generated Answer:", answer)
    


### Answer sample questions

These questions may require filtering and sorting in addition to regular search

In [16]:
answer, query_options, results = answer_query("Who is the youngest employee?")
print_answer(answer, query_options, results)

Generated Query Options: {'orderBy': 'Age asc', 'top': 1}
Search Results
|   ID |   Age | Name       | Title        | Description                      |
|-----:|------:|:-----------|:-------------|:---------------------------------|
|    2 |    20 | Jane Smith | Veterinarian | Provides medical care to animals |
Generated Answer: Based on the provided table, the youngest employee is Jane Smith, who is 20 years old.


In [17]:
answer, query_options, results = answer_query("How many employees are over the age of 25?")
print_answer(answer, query_options, results)

Generated Query Options: {'filter': 'Age gt 25'}
Search Results
|   ID |   Age | Name               | Title                    | Description                             |
|-----:|------:|:-------------------|:-------------------------|:----------------------------------------|
|    5 |    59 | Emily Davis        | Conservation Specialist  | Works on wildlife conservation projects |
|   14 |    32 | William Harris     | Marketing Coordinator    | Promotes zoo events and activities      |
|   17 |    59 | Mia Garcia         | Administrative Assistant | Supports administrative tasks           |
|   19 |    44 | Charlotte Robinson | Event Planner            | Plans and coordinates events            |
|    6 |    29 | Michael Miller     | Zoologist                | Studies animal behavior                 |
Generated Answer: Based on the table, the following employees are over the age of 25:

1. Emily Davis (Age 59)
2. William Harris (Age 32)
3. Mia Garcia (Age 59)
4. Charlotte Robinson (Age

In [18]:
answer, query_options, results = answer_query("Who is the youngest employee over the age of 25?")
print_answer(answer, query_options, results)

Generated Query Options: {'filter': 'Age gt 25', 'orderBy': 'Age asc', 'top': 1}
Search Results
|   ID |   Age | Name          | Title              | Description            |
|-----:|------:|:--------------|:-------------------|:-----------------------|
|   11 |    26 | Olivia Thomas | Facilities Manager | Manages zoo facilities |
Generated Answer: The youngest employee over the age of 25 is Olivia Thomas, who is 26 years old.


In [19]:
answer, query_options, results = answer_query("What do zoo employees do?")
print_answer(answer, query_options, results)    

Generated Query Options: {'search': 'zoo employees job roles'}
Search Results
|   ID |   Age | Name           | Title                 | Description                        |
|-----:|------:|:---------------|:----------------------|:-----------------------------------|
|   11 |    26 | Olivia Thomas  | Facilities Manager    | Manages zoo facilities             |
|   14 |    32 | William Harris | Marketing Coordinator | Promotes zoo events and activities |
|    4 |    23 | Robert Brown   | Tour Guide            | Guides visitors through the zoo    |
|   10 |    43 | James Anderson | Groundskeeper         | Maintains zoo grounds              |
|    3 |    23 | Alice Johnson  | Animal Trainer        | Trains animals for performances    |
Generated Answer: Based on the information provided in the table:

1. **Olivia Thomas (Facilities Manager)**: Manages zoo facilities.
2. **William Harris (Marketing Coordinator)**: Promotes zoo events and activities.
3. **Robert Brown (Tour Guide)**: Guides

In [20]:

try:
    answer, query_options, results = answer_query("Which employee is the most intelligent?")
    print_answer(answer, query_options, results)    
except Exception as e:
    if hasattr(e, 'message'):
        print(e.message)
    else:
        print(e)

Data search cannot be completed as field 'IQ' is not sortable field in the index and is needed by the prompt.
